Inference and qualitative evaluation on custom sentences or dataset samples.
The process demonstrates how to run translation using the best saved checkpoint from a trained from-scratch Transformer.

In [ ]:
# !pip install torchmetrics

In [1]:
import torch
import torchmetrics
from config import get_config
from utils import get_model
from train import get_ds, run_validation
from translate import translate

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

config = get_config()

Using device: cuda


In [3]:
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

checkpoint_path = "results/transformer_weights/tmodel__best.pt"
# checkpoint_path = "/content/tmodel__best.pt"
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)

model.load_state_dict(checkpoint["model_weights"])

<All keys matched successfully>

In [5]:
model.eval()
run_validation(model, val_dataloader, tokenizer_tgt, config["seq_len"], device, lambda msg: print(msg), global_step=0, writer=None, num_examples=10)

----------------------------------------
    SOURCE: Ο Αγαθούλης, η Κυνεγόνδη κ' η γριά ήσαν τώρα στη μικρή πόλη Αβασένα, ανάμεσα στα βουνά της Σιέρρα Μορένα και μιλούσαν ως εξής μέσα σε μια ταβέρνα.
    TARGET: Candide, Cunegund, and the old woman, had by this time reached the little town of Avacena, in the midst of the mountains of Sierra Morena, and were engaged in the following conversation in an inn, where they had taken up their quarters.
 PREDICTED: Candide , who had been by the old woman , had been by the of , and , were so , where they had been so ugly , in the old woman had been brought by a manner of their quarters .
----------------------------------------
    SOURCE: Αυτός ο γιατρός ήτανε ο πιο άσχημος απ' όλους τους άντρες κ' εγώ η πιο δυστυχισμένη απ' όλα τα πλάσματα, αφού έτρωγα ξύλο για έναν άντρα, που δεν τον αγαπούσα.
    TARGET: The doctor himself was the most ugly of all mortals, and I the most wretched creature existing, to be continually beaten for a man whom I d

/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(


0.07234422862529755

In [12]:
from translate import translate

In [13]:
translate("Πλήρωσε αμέσως τη ξαγορά του βαρώνου και του Παγγλώσση.")

SOURCE:    Πλήρωσε αμέσως τη ξαγορά του βαρώνου και του Παγγλώσση.
PREDICTED: Pangloss , by the Baron of Thunder - ten - tronckh , and he was in the Baron ' s body ; and I am more than to Pangloss .
RAW OUTPUT IDS: [2, 63, 4, 47, 5, 132, 6, 451, 54, 221, 54, 475, 4, 8, 18, 15, 12, 5, 132, 50, 64, 304, 13, 8, 14, 87, 75, 82, 7, 63, 9, 3]
Decoding time:     0.317 sec
End-to-end time:   1.159 sec


"Pangloss , by the Baron of Thunder - ten - tronckh , and he was in the Baron ' s body ; and I am more than to Pangloss ."

In [15]:
translate(42)

ID:        42
TARGET:    "You are perfectly right, gentlemen," said Candide, "this is precisely the doctrine of Master Pangloss; and I am convinced that everything is for the best."
SOURCE:    — Έχετε δίκαιο, είπεν ο Αγαθούλης· αυτό μούλεγε πάντα ο κύριος Παγγλώσσης και παρατηρώ καλά, πως όλα είναι άριστα.
PREDICTED: " But , Reverend Father ," said Candide , " this is the best of this world is true , for I am convinced that everything is for the best ."
RAW OUTPUT IDS: [2, 11, 117, 4, 346, 302, 25, 26, 19, 4, 11, 37, 22, 5, 151, 6, 37, 98, 22, 252, 4, 29, 14, 87, 1026, 16, 174, 22, 29, 5, 151, 23, 3]
Decoding time:     0.305 sec
End-to-end time:   1.899 sec


'" But , Reverend Father ," said Candide , " this is the best of this world is true , for I am convinced that everything is for the best ."'